# Questions
1. Why is the context length set to 2048? Look up hugging face documentation
2. What is the number you put in for 'xquery = 20100000' what is it related to?

# Training a GPT-2 Language Model on the POP909 Dataset

The below script is dedicated to training a language model on the POP909 dataset using the GPT-2 architecture. It begins by installing necessary libraries and importing required modules. Then it loads the preprocessed dataset and tokenizes it using the Hugging Face tokenizer.

The script proceeds to define the GPT-2 model architecture and configure it based on specified parameters such as the number of layers, attention heads, and embedding dimension. It calculates and prints the size of the GPT-2 model in terms of parameters.

Next, it prepares the data for training by creating a data collator that handles batch preparation and creates language model labels. It applies the data collator to a small subset of the training data to verify its functionality.

The script then sets up the training environment, including logging configurations for Weights & Biases (WandB) integration and Hugging Face Hub login. It defines a custom trainer class that extends the Trainer class provided by the transformers library. This custom trainer includes additional functionality to log prediction distributions during evaluation.

Training hyperparameters and configuration parameters are defined, and a WandB run is initiated to monitor training progress and log metrics.

The model training loop is executed using the Trainer object, which handles training epochs, batch processing, and evaluation. During evaluation, the custom trainer logs generated audio samples for qualitative analysis.

Once training is complete, the WandB run is finished, and the model checkpoint is saved to the specified output directory. Overall, this notebook provides a comprehensive pipeline for training a GPT-2 language model on the POP909 dataset, including data preprocessing, model setup, training, and result logging.

In [ ]:
import locale
print(locale.getpreferredencoding())

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [27]:
## Install necessary libraries
# !pip install datasets
# !pip install wandb
# !pip install note_seq
# !pip install transformers[torch]
#!pip install accelerate -U

In [ ]:
# Check if the code is running in Google Colab environment
if "google.colab" in str(get_ipython()):
    # Inform the user about installing dependencies in Colab
    print("Installing dependencies...")

    # Install fluidsynth and its development libraries using pip
    !apt-get install fluidsynth
    !apt-get install -qq libasound2-dev libjack-dev

    # Install the pyfluidsynth library using pip
    !pip install -qU pyfluidsynth

In [2]:
# Import necessary libraries and modules
import os
from argparse import Namespace

import note_seq
import numpy as np
import wandb
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoConfig, GPT2LMHeadModel, DataCollatorForLanguageModeling, set_seed, Trainer, TrainingArguments

import matplotlib.pyplot as plt


C:\Users\naomi\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
# Set the Protocol Buffers Python implementation to "python"
# This line is used to resolve compatibility issues related to Protocol Buffers (protobuf)
# It explicitly selects the pure Python implementation of the protobuf library
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [4]:
# Set parameters for WandB (Weights & Biases) integration
wandb_project = "pop909_musicgen"
entity = "musicgen"
data_processed = "pop909_processed"

## Download Dataset and tokenizer from Hugging Face

In the pretokenization notebook, we trained a tokenizer. We'll use it here first to do some basic EDA to understand our data and what type of model size is better (number of layers, heads, etc.)

In [5]:
# Load a dataset named "aimusicgen/pop909_clean_data" using the Hugging Face datasets library
# The split parameter is set to "train" to load the training split of the dataset
ds = load_dataset("aimusicgen/pop909_clean_data", split="train")

# Split the loaded dataset into training and testing sets using train_test_split method
# The test_size parameter specifies the fraction of the dataset to include in the test split (here, 10%)
# The shuffle parameter is set to True to shuffle the data before splitting
raw_datasets = ds.train_test_split(test_size=0.1, shuffle=True)

# Instantiate a tokenizer using the AutoTokenizer class from the transformers library
# The "tokenizer" argument is the pretrained tokenizer from the previous notebook
tokenizer = AutoTokenizer.from_pretrained("aimusicgen/pop909_tokenizer")

# Display the raw datasets, which now include both training and testing splits
# The raw_datasets variable now contains a tuple with two elements: training and testing datasets
raw_datasets


Found cached dataset parquet (C:/Users/naomi/.cache/huggingface/datasets/aimusicgen___parquet/aimusicgen--pop909_clean_data-8139a41134aae9f9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 29930
    })
    test: Dataset({
        features: ['text'],
        num_rows: 3326
    })
})

### Why is the data suffled before it is split into training and testing?

The data is suffled to help introduce **randomness** into the data. This is important because it prevents any inherent order in the dataset from influencing the learning algorithm. If the data is ordered in a certain way (e.g., all samples of one class followed by another), shuffling helps ensure that the model sees a representative mix of samples from all classes during both training and testing.

Some algorithms might perform differently or learn **biased** patterns if trained on data with a specific order. By shuffling the data, you reduce the risk of the model learning patterns based on the order of the examples.

Shuffling contributes to better **generalization** of the model. If the model is exposed to diverse examples during training (rather than learning specific patterns related to the order of the data), it is more likely to perform well on new, unseen data.

When splitting a dataset into training and testing sets, shuffling ensures that both sets contain a representative mix of examples. This is important, especially in scenarios like **cross-validation**, where you repeatedly split the data into different training and testing sets.

In [33]:
# Define the context length for tokenization
# In this case, it is set to 512, meaning the input sequences will be truncated or padded to this length
context_length = 512

# Define a tokenization function named "tokenize" that takes an element as input
def tokenize(element):
    # Use the tokenizer to process the "text" field of the input element
    # Set truncation to True to truncate sequences longer than the specified context length
    # Set max_length to the context_length to ensure all sequences have the same length
    # Set padding to False to avoid adding padding tokens to the sequences
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_length,
        padding=False
    )
    
    # Return a dictionary containing the "input_ids" field from the tokenizer outputs
    return {"input_ids": outputs["input_ids"]}


In [34]:
# Select an example from the training split of the raw dataset (index 1000)
selected_example = raw_datasets["train"][1000]

# Tokenize the selected example using the tokenize function defined earlier
# The tokenize function processes the "text" field of the input example
# and returns a dictionary containing the "input_ids" field with tokenized representation
tk_sample = tokenize(selected_example)

In [35]:
# Print the length of the 'input_ids' field in the tokenized sample
# This indicates the number of tokens in the tokenized representation of the input text
print(f"Len of tk_sample ids {len(tk_sample['input_ids'])}")

# Print the entire tokenized sample
# The tokenized sample is a dictionary containing the 'input_ids' field
print(f"tk_sample {tk_sample}")

Len of tk_sample ids 413
tk_sample {'input_ids': [74, 24, 22, 9, 5, 47, 10, 53, 61, 98, 11, 52, 60, 82, 46, 11, 49, 10, 19, 14, 18, 7, 30, 11, 29, 47, 6, 46, 11, 53, 10, 61, 52, 11, 60, 6, 59, 36, 47, 88, 7, 48, 81, 97, 13, 58, 6, 35, 46, 6, 59, 36, 47, 41, 58, 35, 7, 46, 87, 8, 9, 5, 59, 55, 38, 65, 41, 58, 54, 37, 7, 38, 55, 59, 12, 64, 6, 37, 14, 54, 14, 58, 14, 30, 47, 49, 53, 98, 5, 29, 46, 48, 52, 97, 8, 9, 5, 71, 65, 10, 78, 32, 70, 10, 71, 38, 6, 77, 11, 55, 55, 70, 11, 54, 54, 59, 59, 13, 58, 61, 6, 37, 7, 31, 64, 15, 60, 53, 65, 53, 47, 58, 5, 64, 52, 11, 46, 52, 8, 9, 5, 47, 47, 36, 7, 46, 7, 35, 46, 61, 53, 69, 11, 52, 60, 71, 19, 10, 70, 61, 36, 10, 60, 47, 61, 10, 46, 53, 12, 35, 7, 18, 52, 59, 6, 68, 12, 60, 58, 55, 59, 36, 94, 11, 58, 6, 69, 10, 40, 11, 35, 19, 13, 32, 6, 18, 7, 93, 68, 6, 31, 6, 39, 14, 54, 8, 23, 24, 22, 9, 5, 47, 10, 53, 61, 98, 11, 52, 60, 82, 46, 11, 49, 10, 19, 14, 18, 7, 30, 11, 29, 47, 6, 46, 11, 53, 10, 61, 52, 11, 60, 6, 59, 36, 47, 88, 7, 48,

As you can see only 742 tk_sample ids came back when there was an index of 1,000. This is possibly due to padding. It was set to *padding=False* in the *tokenize* function. Without padding, the resulting *input_ids* will not be padded to the maximum length, and if the original text is shorter than the specified *max_length*, the tokenized sequence will be shorter.  

In [36]:
# Apply the tokenize function to the entire raw dataset using the map method
# The tokenize function is applied in a batched manner, improving efficiency
# The remove_columns parameter is set to remove the columns from the raw dataset 
# (excluding the "train" split) after tokenization, as they are no longer needed
tokenized_datasets = raw_datasets.map(
    tokenize,                # The tokenization function to be applied
    batched=True,            # Tokenize in batches for efficiency
    remove_columns=raw_datasets["train"].column_names  # Remove unnecessary columns after tokenization
)

# Display the resulting tokenized datasets
# The tokenized_datasets variable now contains both training and testing splits with tokenized representations
tokenized_datasets

Map:   0%|          | 0/29930 [00:00<?, ? examples/s]

Map:   0%|          | 0/3326 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 29930
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 3326
    })
})

## Train the model

In [41]:
# Define hyperparameters for model architecture
# These parameters can be adjusted based on the size of the data and the specific requirements of the task

n_layer = 6 # Number of layers in the transformer model
n_head = 8 # Number of attention heads in each transformer layer
n_emb = 512 # Dimensionality of the embedding layer

In [42]:
# Use Hugging Face's AutoConfig to create a configuration for the GPT-2 model
# The configuration is based on the "gpt2" pre-trained model, and some parameters are customized

# Define the configuration using AutoConfig
config = AutoConfig.from_pretrained(
    "gpt2",                               # Base model: "gpt2"
    vocab_size=len(tokenizer),            # Vocabulary size based on the tokenizer
    n_positions=context_length,           # Maximum position embeddings (context length)
    n_layer=n_layer,                      # Number of transformer layers
    n_head=n_head,                        # Number of attention heads in each layer
    pad_token_id=tokenizer.pad_token_id,  # ID of the padding token
    bos_token_id=tokenizer.bos_token_id,  # ID of the beginning-of-sequence token
    eos_token_id=tokenizer.eos_token_id,  # ID of the end-of-sequence token
    n_embd=n_emb                           # Dimensionality of the embedding layer
)

# Set the num_attention_heads after creating the configuration
    #config.num_attention_heads = 0

# Create an instance of the GPT-2 language model using the configured parameters
model = GPT2LMHeadModel(config)


In [43]:
# Calculate and print the size of the GPT-2 model in terms of parameters
# The size is measured in millions (M) of parameters

# Calculate the total number of parameters in the GPT-2 model
model_size = sum(t.numel() for t in model.parameters())

# Print the size of the GPT-2 model in millions of parameters
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 20.1M parameters


Create a datacollator to will take care of preparing the data:

"Before we can start training, we need to set up a data collator that will take care of creating the batches. We can use the DataCollatorForLanguageModeling collator, which is designed specifically for language modeling (as the name subtly suggests). Besides stacking and padding batches, it also takes care of creating the language model labels — in causal language modeling the inputs serve as labels too (just shifted by one element), and this data collator creates them on the fly during training so we don’t need to duplicate the input_ids.

Note that DataCollatorForLanguageModeling supports both masked language modeling (MLM) and causal language modeling (CLM). By default it prepares data for MLM, but we can switch to CLM by setting the argument mlm=False:"

In [44]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [45]:
# Test the data collator on a small subset of the training data
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])

# Print the shapes of the collated outputs
for key in out:
    print(f"{key} shape: {out[key].shape}")

# Display the collated outputs
print(f"Collated outputs: {out}")

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids shape: torch.Size([5, 359])
attention_mask shape: torch.Size([5, 359])
labels shape: torch.Size([5, 359])
Collated outputs: {'input_ids': tensor([[ 74,  24,  22,  ...,   3,   3,   3],
        [ 74,  24,  22,  ...,   3,   3,   3],
        [ 74,  24,  22,  ..., 111,   8,  23],
        [ 74,  24,  22,  ...,   3,   3,   3],
        [ 74,  24,  22,  ...,   3,   3,   3]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[  74,   24,   22,  ..., -100, -100, -100],
        [  74,   24,   22,  ..., -100, -100, -100],
        [  74,   24,   22,  ...,  111,    8,   23],
        [  74,   24,   22,  ..., -100, -100, -100],
        [  74,   24,   22,  ..., -100, -100, -100]])}


In [46]:
# Login into wandb
wandb.login()

wandb: Currently logged in as: naomitunstead (musicgen). Use `wandb login --relogin` to force relogin


True

In [47]:
# Set the WandB environment variable to log the model checkpoint
%env WANDB_LOG_MODEL='checkpoint'

env: WANDB_LOG_MODEL='checkpoint'


In [48]:
# Login into Hugging Face
notebook_login()

In [49]:
# Create the args for out trainer

# Get the output directory with timestamp.
output_path = "output"
steps = 5000

# Define training configuration parameters
# Commented parameters correspond to the small model
config = {"output_dir": output_path,
          "num_train_epochs": 1,
          "per_device_train_batch_size": 8,
          "per_device_eval_batch_size": 4,
          "evaluation_strategy": "steps",
          "save_strategy": "steps",
          "eval_steps": steps,
          "logging_steps":steps,
          "logging_first_step": True,
          "save_total_limit": 5,
          "save_steps": steps,
          "lr_scheduler_type": "cosine",
          "learning_rate":5e-4,
          "warmup_ratio": 0.01,
          "weight_decay": 0.01,
          "seed": 1,
          "load_best_model_at_end": True,
          "report_to": "wandb"}

# Create Namespace object with configuration
args = Namespace(**config)

In [50]:
# Set the random seed for reproducibility
set_seed(args.seed)

In [51]:
# Initialize and start a new WandB run for training
run = wandb.init(project=wandb_project, job_type="training", config=args)

In [52]:
# Code for converting token sequences to NoteSequences with audio-related information

# Constants for note durations
NOTE_LENGTH_16TH_120BPM = 0.25 * 60 / 120
BAR_LENGTH_120BPM = 4.0 * 60 / 120

def token_sequence_to_note_sequence(token_sequence, use_program=True, use_drums=True, instrument_mapper=None, only_piano=False):
    """
    Convert a token sequence to a NoteSequence with audio-related information.
    Args:
        token_sequence (list or str): Token sequence representing musical information.
        use_program (bool): Whether to use program information for instruments.
        use_drums (bool): Whether to include drums in the output.
        instrument_mapper (dict): Mapping of instrument names to MIDI program numbers.
        only_piano (bool): Whether to include only piano instruments in the output.
    Returns:
        note_sequence (NoteSequence): Converted NoteSequence.
    """

    if isinstance(token_sequence, str):
        token_sequence = token_sequence.split()

    note_sequence = empty_note_sequence()

    # Render all notes.
    current_program = 1
    current_is_drum = False
    current_instrument = 0
    track_count = 0
    for token_index, token in enumerate(token_sequence):

        if token == "PIECE_START":
            pass
        elif token == "PIECE_END":
            print("The end.")
            break
        elif token == "TRACK_START":
            current_bar_index = 0
            track_count += 1
            pass
        elif token == "TRACK_END":
            pass
        elif token == "KEYS_START":
            pass
        elif token == "KEYS_END":
            pass
        elif token.startswith("KEY="):
            pass
        elif token.startswith("INST"):
            instrument = token.split("=")[-1]
            if instrument != "DRUMS" and use_program:
                if instrument_mapper is not None:
                    if instrument in instrument_mapper:
                        instrument = instrument_mapper[instrument]
                current_program = int(instrument)
                current_instrument = track_count
                current_is_drum = False
            if instrument == "DRUMS" and use_drums:
                current_instrument = 0
                current_program = 0
                current_is_drum = True
        elif token == "BAR_START":
            current_time = current_bar_index * BAR_LENGTH_120BPM
            current_notes = {}
        elif token == "BAR_END":
            current_bar_index += 1
            pass
        elif token.startswith("NOTE_ON"):
            pitch = int(token.split("=")[-1])
            note = note_sequence.notes.add()
            note.start_time = current_time
            note.end_time = current_time + 4 * NOTE_LENGTH_16TH_120BPM
            note.pitch = pitch
            note.instrument = current_instrument
            note.program = current_program
            note.velocity = 80
            note.is_drum = current_is_drum
            current_notes[pitch] = note
        elif token.startswith("NOTE_OFF"):
            pitch = int(token.split("=")[-1])
            if pitch in current_notes:
                note = current_notes[pitch]
                note.end_time = current_time
        elif token.startswith("TIME_DELTA"):
            delta = float(token.split("=")[-1]) * NOTE_LENGTH_16TH_120BPM
            current_time += delta
        elif token.startswith("DENSITY="):
            pass
        elif token == "[PAD]":
            pass
        else:
            #print(f"Ignored token {token}.")
            pass

    # Make the instruments right.
    instruments_drums = []
    for note in note_sequence.notes:
        pair = [note.program, note.is_drum]
        if pair not in instruments_drums:
            instruments_drums += [pair]
        note.instrument = instruments_drums.index(pair)

    if only_piano:
        for note in note_sequence.notes:
            if not note.is_drum:
                note.instrument = 0
                note.program = 0

    return note_sequence

def empty_note_sequence(qpm=120.0, total_time=0.0):
    """
    Create an empty NoteSequence with specified tempo and total time.
    Args:
        qpm (float): Quarter notes per minute (tempo).
        total_time (float): Total time of the NoteSequence.
    Returns:
        note_sequence (NoteSequence): Empty NoteSequence.
    """
    note_sequence = note_seq.protobuf.music_pb2.NoteSequence()
    note_sequence.tempos.add().qpm = qpm
    note_sequence.ticks_per_quarter = note_seq.constants.STANDARD_PPQ
    note_sequence.total_time = total_time
    return note_sequence

In [53]:
# first create a custom trainer to log prediction distribution
# Set the sample rate for audio processing
SAMPLE_RATE=44100

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def evaluation_loop(
        self,
        dataloader,
        description,
        prediction_loss_only=None,
        ignore_keys=None,
        metric_key_prefix="eval",
    ):
        # Call super class method to get the eval outputs
        eval_output = super().evaluation_loop(
            dataloader,
            description,
            prediction_loss_only,
            ignore_keys,
            metric_key_prefix,
        )

         # Log the prediction distribution using `wandb.Histogram` method.
        if wandb.run is not None:
            # Encode a starting token to begin the generation
            input_ids = self.tokenizer.encode("PIECE_START", return_tensors="pt").cuda()

            # Generate more tokens for each voice
            for voice_num in range(1, 5):
                generated_ids = self.model.generate(
                    input_ids,
                    max_length=2048,
                    do_sample=True,
                    temperature=0.75, # Set temperature for sampling (higher values for more randomness, lower for more determinism)
                    #top_p = 0.9, # Set top-p sampling parameters (nucleus sampling) to control diversity
                    #top_k = 50, # Set top-k sampling parameters to restrict generation to the top-k most likely tokens
                    eos_token_id=self.tokenizer.encode("TRACK_END")[0]
                )

                # Decode the generated tokens into a token sequence
                token_sequence = self.tokenizer.decode(generated_ids[0])

                # Convert the token sequence into a NoteSequence
                note_sequence = token_sequence_to_note_sequence(token_sequence)

                # Synthesize the audio from the NoteSequence
                synth = note_seq.fluidsynth
                array_of_floats = synth(note_sequence, sample_rate=SAMPLE_RATE)

                # Convert the float audio samples to int16 format
                int16_data = note_seq.audio_io.float_samples_to_int16(array_of_floats)

                # Log the generated audio using the wandb.Audio method
                wandb.log({"Generated_audio_voice_" + str(voice_num): wandb.Audio(int16_data, SAMPLE_RATE)})

        # Return the evaluation output
        return eval_output

In [54]:
# Create TrainingArguments object with training configuration 
train_args = TrainingArguments(**config)

In [55]:
 # Create training arguments
train_args = TrainingArguments(**config)

# Initialize the custom trainer for training the model
trainer = CustomTrainer(
    model=model,
    tokenizer=tokenizer,
    args=train_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    # compute_metrics=compute_metrics_fn
)

In [56]:
# Train the model.
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# call wandb.finish() to finish the run
wandb.finish()

Model Performance Evaluation: Validation and training loss provide insights into how well your model is performing during training and validation phases. High training loss might indicate that the model is not learning properly, while high validation loss might indicate overfitting or poor generalization.

Overfitting Detection: Monitoring the difference between training and validation loss helps you detect overfitting. If your model performs well on the training data but poorly on the validation data, it's a sign that the model is overfitting to the training data and not generalizing well.

Hyperparameter Tuning: Understanding the loss metrics helps in tuning hyperparameters like learning rate, batch size, and regularization techniques. Adjusting these parameters based on the loss metrics can improve model performance.

Model Selection: Comparing validation loss across different models helps in selecting the best-performing model for deployment. Lower validation loss generally indicates a better-performing model.

Early Stopping: Monitoring the validation loss can help implement early stopping strategies. When validation loss stops improving or starts to increase, it's an indication to stop training, preventing the model from overfitting further.

Feedback Loop: Validation loss provides feedback on the effectiveness of changes made to the model architecture, data preprocessing, or other components. This iterative process helps in refining the model and improving its performance.

Communication: Reporting validation and training loss is standard practice in research and applied deep learning projects. It enables clear communication of the model's performance and its behavior during training.

Overall, understanding and monitoring validation and training loss are essential for ensuring the effectiveness, generalization, and performance of your deep learning models.

Decreasing Loss: During training, the loss should generally decrease over time. This indicates that the model is learning and adjusting its parameters to better fit the training data.

Convergence: Ideally, both training and validation losses should converge to a stable value. This suggests that the model has learned as much as it can from the available data.

Training Loss: It's common for the training loss to be lower than the validation loss. This is because the model is optimized specifically for the training data. However, if the training loss is significantly lower than the validation loss, it could indicate overfitting.

Validation Loss: The validation loss should be relatively low, indicating that the model generalizes well to unseen data. If the validation loss is substantially higher than the training loss, it suggests overfitting.

Baseline Comparison: Compare your model's performance with a baseline model or previous results in the literature if available. This gives you a reference point for evaluating the adequacy of your model's performance.

Domain Knowledge: Consider the context of your problem and what level of performance is practically useful. In some cases, even relatively high losses may be acceptable if the model achieves the desired outcome.

Regularization: Regularization techniques like dropout, weight decay, or early stopping can help control overfitting and improve generalization, leading to lower validation loss.